# Week 8 Overview
Matching is a powerful alternative to regression for estimating treatment effects when linear assumptions may not hold or when the relationship between variables is complex. This content introduces the concept of matching as a way to “close back doors” by pairing treated and untreated units with similar covariates, enabling more accurate estimation of causal effects. You’ll explore different matching techniques, including distance-based matching, propensity score matching, inverse probability weighting, and kernel-based weighting. The content also covers how to match using one or multiple variables, how to assess balance and common support, and how to handle practical challenges like bias-variance tradeoffs and the curse of dimensionality. By the end, you’ll understand how matching can be used on its own or combined with regression to support valid causal inference. 

## Learning Objectives 
At the end of this week, you will be able to: 
- Explain matching and its application to average treatment effect (ATE), ATT (average treatment effect on the treated), and ATUT (average treatment effect on the untreated) 
- Perform inverse probability weighting and propensity score matching  
- Explain matching variants such as the Mahalanobis distance and the Epanechnikov kernel 

## Topic Overview: Matching Fundamentals and Strategies
This section introduces matching as a flexible alternative to regression for closing back doors when the relationship between variables is complex or nonlinear. 

Instead of modeling the functional form directly, **matching** pairs treated and untreated observations with similar values of a confounding variable, $Z$. 

This allows estimation of treatment effects, like ATT, even without knowing the true form of the relationship between treatment and outcome. 

The section also covers **weighted matching** as a way to account for distributional imbalances and introduces key concepts like distance matching and propensity score matching, which help formalize similarity between observations using one or more covariates. 

### Learning Objectives 
- Explain matching and its application to ATE, ATT, and ATUT 
- Perform inverse probability weighting and propensity score matching 

### 1.1 Lesson: Another Way to Close Back Doors
What if the relationship between treatment and outcome isn’t something we can easily model with a straight line? In this video, we’ll explore how **matching** can help us estimate treatment effects—even when the relationship between variables is complex, messy, or nonlinear. 

#### Matching
With regression, we close back doors by finding a linear relationship between the outcome and the treatment, as well as with any confounders. But what if there isn’t a linear relationship, and we’re not sure how to model the situation? 

For example, what if:

$$ Y = \frac{2X + X^2}{Z} + \varepsilon(X, Z) $$

Then, we *could* model this linearly by including the term $\frac{X^2}{Z}$  as an additional covariate, but what if we don’t think to do this? It’s a pretty weird relationship — we’re unlikely to think of it unless our domain knowledge hints of it:

One solution would be matching: 

1. For each item with $X = 1$, we note its $Z$ value.
2. Then we find another item with approximately the same $Z$ value, and match them. 

Suppose the $X = 0$ values have $Z = 1, 2$, while the $X = 1$ values have $Z = 1, 2, 3$. We throw out the $Z = 3$ item (it doesn't match), and we get: 

$Y(X = 0, Z = 1) \; \& = 0$

$Y(X = 0, Z = 2) \; \& = 0$

$Y(X = 1, Z = 1) \; \& = 3$

$Y(X = 1, Z = 1) \; \& = 2.5$

In this scenario the estimated effect is the difference between the average $X = 1$ and $X = 0$ values, or 

$$  \frac{ (\frac{3. + 2.5}{2} - \frac{0}{2} )}{2} \; = \; 1.375$$

This is the average treatment effect on the treated, since we are including counterfactuals (matches) for all treated items, but we are not including counterfactuals for all untreated items ($Z = 3$ has no counterfactual). Matching works even though we do not know the true relationship between $X$, $Y$, and $Z$. All we need to know is that $Z$ is the only confounder. 

### 1.2 Lesson: Weighted Averages
We can also match by **weighting** 

In this case, if the untreated group has $Z = 1, 2, 3$ while the treated group has $Z = 1, 2$ 

Then we could weight the untreated values according to how close they are to the treated values: 
- For the untreated items, maybe the $Z = 1$ item would have a weight of 0.4, 
- $Z = 2$ has $0.4$, and 
- $Z = 3$ has 0.2 because $Z = 3$ is farther away from the treated $Z$ values. 

The exact weight is somewhat arbitrary: 
- Of the untreated items, $Z = 3$ should get a lower weight than $Z = 2$ because it’s farther from the treatment samples’ $Z$ values, but what weight, exactly, should it get?  

There are many ways of calculating this, but in the end, it’s up to you. There is no one right answer. 
- For the treated items" $Z = 1$ and $Z = 2$ both get an equal weight of $0.5$ 
- Then, to compute the effect, we weight each $Y$ value by the associated weight based on the $Z$ value. 

Another example could be if $Z$ is categorical: 

There are men and women. 
- If there are more men in the untreated group than in the treated group, we should downweight the untreated men (say, before computing the mean of ﻿Y﻿ or other statistic) according to the ratio of the count.
- If there are also more women in the untreated group, we would downweight the untreated women, too, but likely by a different amount. 
- if the treated group has 8 men and 2 women, while the untreated group has 4 men and 6 women, then we could compute the ATT by weighting the untreated group so that men get a total weight of 0.8 and women 0.2. - This would involve weighting men at 0.2 each and women at 0.0333 each. The treated group would weight everyone at 0.1. In this way, we effectively have one counterfactual for each man and woman in the treated group. 

Taken together, these weights would allow us to compute the mean or other statistic of the untreated group as if it had the same distribution of men and women as the treated group. 

#### A Single Matching Variable
As noted above, there’s a significant problem with trying to match the “nearest” $Z$ value between treated and untreated groups: 
- We need to know which $Z$ value is closest (and there may be multiple ways of measuring that). 
- When we are weighting, we need to know what weights to assign based on how close the $Z$ value is to other $Z$ values.

The simplest approach is **distance matching:**
- Distance Matching matches east treatment item to the closest untreated item(s) in distance, according ot the proximity of the matching variables (i.e., confounders like $Z$).
- If there is one matching variable only, this is easy; but if there are several matching variables, we have to define a distance between samples where there are multiple covariates ($Z_1, Z_2, Z_3,$ etc.)
- One way would be to standardize them, then take the euclidian distance, like $ \sqrt{(Z_{1,T} - Z_{1, U})^2 + (Z_{2,T} - Z_{2, U})^2 + (Z_{3,T} - Z_{3, U})^2}$.
    - Standardization is important because if the difference for $Z_1$ are on the order of 100, but for $Z_2$they are on the order of 1, then the $Z_1$ differences will completely outweight the $Z_2$ differences.
    - This is especially true with Euclidean distances!, $\sqrt{100^2 + 1^2}$  is $100.005$ ; the $1$ doesn't really matter because of the squaring.

Perhaps the most common approach is **propensity score matching:**
- In propensity score matching observations are similar if they were equally likely to be treated. 
- For example: 
    - if the matching variable is income, and the likelihood of treatment is normally distributed with mean $50,000, then the values $40,000 and $60,000 are “similar” because they are equally likely to be treated. - So, we can match a treated variable with income $40,000 to an untreated variable with $40,000 or with $60,000. This works to close the back door, although this may not be obvious. 
    
To see why, suppose we have:

In [2]:
import pandas as pd
import numpy as np

dict = {
    'Group' : ['Treated', 'Treated', 'Treated', 'Treated', 'Untreated', 'Untreated', 'Untreated', 'Untreated','Untreated', 'Untreated',],
    'Income' : [40000, 50000, 50000, 60000, 40000, 40000, 50000, 50000, 60000, 60000],
    'Target value' : [12, 18, 18, 0, 12, 12, 18, 18, 0, 0]
    }
df = pd.DataFrame(dict)
df.head()

,Group,Income,Target value
0,Treated,40000,12
1,Treated,50000,18
2,Treated,50000,18
3,Treated,60000,0
4,Untreated,40000,12


Then, taking the overall means results in the treated target value mean=12 and untreated=10:

In [3]:
df.groupby('Group').mean()

,Income,Target value
Group,,
Treated,50000.0,12.0
Untreated,50000.0,10.0


But the income is a *confounder*; the two groups are actually the same when controlling for income. (This must be the case: the incomes’ targets match exactly across both groups.) 

If we match incomes using distance matching, for distance = 0, the untreated group gets 12, 18, 18, 0, and is the same target mean as the control group. 

This would involve the treated item with income $40,000 matching to the average of the two untreated items, with income $40,000 (all three of these have target = 12).

In [5]:
# Create a new dataframe with numeric treatment indicator
df_numeric = df.copy()
df_numeric['Treatment'] = (df_numeric['Group'] == 'Treated').astype(int)

# Separate the data into treated and untreated groups
treated = df_numeric[df_numeric['Treatment'] == 1]
untreated = df_numeric[df_numeric['Treatment'] == 0]

# Create a new dataframe for matched pairs
matched_treated = treated.copy()
matched_untreated = pd.DataFrame(columns=df_numeric.columns)

# For each treated observation, find matching untreated observations
for _, treated_row in treated.iterrows():
    # Find untreated observations with the same income (distance = 0)
    matches = untreated[untreated['Income'] == treated_row['Income']]
    
    # If we found matches, add them to our matched dataset
    if not matches.empty:
        # Calculate the average of the matched untreated observations
        avg_target = matches['Target value'].mean()
        avg_income = matches['Income'].mean()
        
        # Create a new row for the matched untreated observation
        new_row = pd.DataFrame({
            'Group': ['Untreated (Matched)'],
            'Income': [avg_income],
            'Target value': [avg_target],
            'Treatment': [0]
        })
        
        # Add this match to our matched untreated data
        matched_untreated = pd.concat([matched_untreated, new_row], ignore_index=True)

# Combine the matched treated and untreated data
matched_data = pd.concat([matched_treated, matched_untreated], ignore_index=True)

# Display results
print("Original data means by treatment status:")
print(df.groupby('Group')['Target value'].mean())

print("\nMatched data means:")
print("Treated mean:", matched_treated['Target value'].mean())
print("Untreated (matched) mean:", matched_untreated['Target value'].mean())
print("Treatment effect:", matched_treated['Target value'].mean() - matched_untreated['Target value'].mean())

Original data means by treatment status:
Group
Treated      12.0
Untreated    10.0
Name: Target value, dtype: float64

Matched data means:
Treated mean: 12.0
Untreated (matched) mean: 12.0
Treatment effect: 0.0


/var/folders/4k/nl2zz_ln4d55dcscl_vvjglh0000gn/T/ipykernel_79623/373636855.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  matched_untreated = pd.concat([matched_untreated, new_row], ignore_index=True)


But what about propensity score matching? 

This would mean that we can randomly cross-match untreated-$40,000 and untreated-$60,000 to treated-$40,000 and treated-$60,000 if we like because they have the same propensity to be treated (one-third of each are treated, compared with one-half for $50,000). 

This will produce the same target value average in the untreated matched group as for distance matching (in this case, 12). 

The propensity score approach involves treated-$40,000 matching to untreated-$40,000 or untreated-$60,000 and untreated-$60,000 to untreated-$40,000 or untreated-$60,000. In other words, the two untreated values are equally likely to be chosen. 

The distance approach involves treated-$40,000 matching to untreated-$40,000 and treated-$60,000 matching to untreated-$60,000. So again, the two untreated values are equally likely to be chosen (they are chosen once each). 

Then, having a 50% likelihood of picking $40,000 or $60,000 for the match gives the same average target value as always picking exactly one $40,000 and one $60,000. 

An alternative perspective is that the confounder (income) influences the propensity score (which is 33% for the high and low income, 50% for the medium one), which in turn influences the treatment. If we think of this in terms of directed acyclic graphs, we’ll see that the path through income toward the outcome (﻿Y﻿) must pass through the propensity score. Therefore, controlling for the propensity score suffices to control for this path.

## Topic 2: Practical Considerations and Advanced Techniques
This section explains how to calculate propensity scores and use them effectively in matching and weighting strategies for causal inference. 
- You'll learn how logistic regression or binning can estimate treatment probabilities and explore practical decisions like how many matches to select, whether to match with or without replacement, and how to assign weights based on distance using kernels like Epanechnikov. 
- The section also introduces inverse probability weighting (IPW), considerations for caliper selection, matching in high-dimensional spaces, and advanced techniques like Mahalanobis distance, coarsened exact matching, and entropy balancing. 
- Finally, you'll see how to validate your propensity score models through stratification tests and how machine learning can enhance score estimation. 

### Learning Objectives 
- Perform inverse probability weighting and propensity score matching  
- Explain matching variants such as the Mahalanobis distance and the Epanechnikov kernel.

### 2.1 Lesson: Propensity Scores and Matching Strategies
When people receive a treatment, it’s often not random — it’s influenced by their characteristics. In this video, you’ll learn how propensity scores help us account for those differences and how we can use them — along with a method called inverse probability weighting — to estimate causal effects more accurately.

#### How to Calculate Propensity Scores
With one covariate (such as income in the example above), we’d often just match on the value of the covariate itself. But if we wanted to use propensity score, we could compute it in one of two ways:
- Perform logistic regression to model the probability of treatement. Above, we'd want to fit a quadratic function where:
$$\text{Prob} (40000) = \frac{1}{3}, \text{Prob} (50000) = \frac{1}{2}, \text{Prob} (60000) = \frac{1}{3}$$
- It has to be quadratic, including a $Z^2$ term, because we want it to be small for both high and low incomes and larger for the middle incomes.
- Bin the data(in this case, perhaps into bins of 10,000) and compute the probability of treatment in each bin.

#### Selecting Matches or Constructing a Matched Weighted Sample?  
Selecting matches means selecting one or more untreated observations for each treated observation (or vice-versa). 

In a matched weighted sample, each untreated observation is assigned a numerical “weight” that determines how important it is. 

The “selecting matches” effect can vary more with small changes to the dataset as the matched observations can be “in” or “out” according to small differences in $Z$ values. 

Suppose a treated item has $Z = 0.55$, and the two nearest untreated items have $Z = 1$ and $Z = 0$. 

This treated item is closer to $Z = 1$ than to $Z = 0$. But a small change to $Z = 0.45$ can make it closer to $Z = 0$.

#### If We’re Selecting Matches, How Many? 

Suppose we compute the ATT so that we find untreated matches (counterfactuals) for each treated item. There might be multiple good matches for each treated item. What do we do in that case? There are several options. 

1. Pick the best match. 
2. Pick the top k best matches (k-nearest-neighbor matching). We can then average their $Y$ values in order to find the effect. 
3. Pick all the matches within a given distance (radius matching). We can find all untreated $Z$ values within a radius of a given treated $Z$ value. Again, we average their $Y$ values. Radius matching can assign a varying number of matches to each treated item, so we’d want to weight these matches so that the effective number of matches to each treated item is 1. If one treated item has two matches and another has three, we’d weight the former by 0.5 each and the latter by 0.333 each. 

We can also match “with replacement,” meaning that a given untreated observation can match to multiple treated observations, or “without replacement,” meaning that each untreated observation can only match to one treated observation. 

One suggestion: If your untreated group contains many more observations than your treated group, it will make more sense to assign multiple untreated observations to each treated one. If your untreated group contains about the same number of observations as your treated group, you might not have enough observations to do that. 

#### If We’re Constructing a Matched Weighted Sample, How Will Weights Decay with Distance?
In an example earlier, I said that the farther-away $Z = 3$ value would get a weight of $0.2$, while the closer ones got a weight of $0.4$ each. That was just a random guess — is there any more systematic way to assign weights based on distance?

One approach is called the **Epanechnikov kernel**, which assigns a weight given the distance $x:$

$K (x) \; = \; \frac{3}{4} (1 - x^2)$

In this case, since $Z = 3$ was a distance of 1 away from $Z = 2$, we'd have:

$K(3 - 2) \; = \; \frac{3}{4} (1 - (3-2)^2)$

$ = \; \frac{3}{4} (1 - 1) \; = \; 0$ (So the $Z = 3$ value would be too far away to be included). 

If it were $Z = 2.5$ instead, we'd have $K(2.5 - 2) \; = \; \frac{3}{4} (1 - (0.5)^2) \; = \; 0.5625$ as its weight. We might want to rescale the kernel if our typical distances are too large for too small for this kernel to be meaningful. 

The Epanechnikov kernel is similar to doing radius matching with a radius of 1, excep that the weight increases continuously up to this radius rather than assigning a disrete value of either 0 or 1.

### 2.2 Lesson: Advanced Matching and Weighting Techniques, Part I

#### Inverse Probability Weighting (IPW)
This approach is used with propensity scores. 
- The weight for a given observation is equal to one divided by its probability. 
- For a treated observation, we use the probability that it would be treated. 
- For an untreated observation, we use the probability that it would be untreated. 
- This probability could be calculated using either (a) logistic regression based on one or more $Z$ values or (b) if $Z$ is categorical, we can simply count up the treatment probability for the given $Z$.

**"Treatment value”** means **“treated”** or **“untreated":** 
- In other words, if items with income 40,000 have a propensity score of 33% (one-third of them are treated, and two-thirds are untreated) 
- Then they are weighted by $\frac{1}{0.33} = 3$ in the treatment group, and $\frac{1}{0.67} = 1.5$ in the control group.
-  The result is that the total weight in the treatment and untreated group is, on average, the same (it is two times the number of items).
- That's because for (on average) $Np$ items in *treatment* and $N(1 - p)$ items in the untreated grup, the total weight is $N \cdot \frac{p}{p} = N$ in treatement, and $N \frac {1 - p}{1 - p} = N$ in the untreated group. 

In effect, if we have $N$ items of a certain type \(around income \$40,000, or having a similar propensity score to that\), then inverse probability weighting assigns weights so that the total weight of these items is $2N$ , i.e., the average weight of each item is 2. 

What’s meant here is not that the items’ weights must total exactly $2N$ but that if:
- (1) a number of samples were taken from the population and 
- (2) the model for computing the propensity is well-specified, with the correct formula, then on average, the total will be $2N$. 



This means that we are computing the average treatment effect (each item counts for 2, whether it is treated or untreated) instead of the average treatment effect on the treated (each treated item counts for a fixed amount, and untreated items count according to their treated match).

Let’s consider the example above. If items with income 50,000 have a propensity score of 50%, they are weighted by $\frac{1}{0.5} = 2$ in the treatment group and $\frac{1}{1 - 0.5} = 2$ in the control group. In the above example, applying IPW to each sample, this results in a $Y$ value of 

$$\frac {12 \times 3 + 18 \times 2 + 18 \times 2 + 0 \times 3}{10} = 10.8$$

The same $Y$ value for treated and untreated.

Let’s consider this special case where the treated and untreated items have the same target value given the same propensity score. There are $N_T$ of a given matching value in treated, which all have target value $T$ , and $N_U% in untreated, which also have target value ﻿T﻿ , then (assuming the statistical distribution of the data exactly reflects the 


#### What Is the Worst Acceptable Match?
What if one treated observation is totally different from all untreated observations? 
- We define a “caliper” or “bandwidth,” which is a number of standard deviations (either of the propensity score or of the matched value, whichever you are using to match; in the latter case, I am assuming that the matched covariate is a single number ﻿Z﻿ so that it has a single standard deviation). 
- You can use the Epanechnikov kernel, which automatically has a caliper of 1. 
- You can use exact matching, which automatically has a caliper of 0. (This is unsuitable for continuous data values that rarely or never take on the same value.) 
- You can use coarsened exact matching, where you bin the data and then require that the two matches (treated and untreated) be in the same bin. (This is suitable for continuous data values.) 

Recall the **bias-variance tradeoff:** 
- High bias means that our prediction is systematically wrong — even if we took an infinite number of samples and averaged the results, it would still be wrong. 
- High variance means that our prediction varies dramatically when we take different samples. The wider the bandwidth, the more bias because the $Y$ values in the large bin may differ systematically from each other. 
- However, the wider the bandwidth, the less variance because we sample more possible matches. In other words: 
    - if we only take a single $Y$ value from a small bin, it should be correct on average but might be wrong by chance. 
    - If we take many matching $Y$ values from a large bin, the average will be stable, but that average might differ systematically from the correct value. 
    - Another way of putting it is that if the matches are at a distance of 1, 2, 3, 4, and 5 away (in terms of the matching value), then using just the distance 1 value has low bias (because we’re using the best estimator available) but high variance (we’d need to take an average of multiple estimates to reduce the variance.) 

The bias-variance tradeoff is generally important when performing matching! 

